# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
%%time
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';',
                     parse_dates=True, keep_date_col=True)

In [ ]:
%%time
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', 
                        parse_dates=True, keep_date_col=True,)

In [ ]:
azdias.head()

In [ ]:
azdias.shape

In [ ]:
customers.shape

In [ ]:
azdias.info()

In [ ]:
customers.info()

As it can be seen above, reading the csv files takes more than 12 mins. I decided to decrease the memory usage for the files to make the process of data analysis more efficient.

I took the memory usage function from the following [repository](https://github.com/harshdarji23/Arvato-Identifying-the-potential-customers/blob/master/Jupyter%20Notebook/Arvato%20Project%20Workbook%20-Final.ipynb), which was created from this [source code from Kaggle](https://www.kaggle.com/gemartin/load-data-reduce-memory-usage).

In [ ]:
def reduce_mem_usage(df):
    """ This function iterates through all the columns of a dataframe and modifies the data type
        to reduce memory usage. 
    Parameters:
    df (pandas DataFrame): a dataframe, which needs to be reduced
    
    Returns:
    df (pandas DataFrame): a dataframe with a reduced memory usage
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time
azdias_red_mem = reduce_mem_usage(azdias)

In [ ]:
%%time
customers_red_mem = reduce_mem_usage(customers)

Let us save the optimized files

In [ ]:
azdias_red_mem.to_csv('Udacity_AZDIAS_reduced_memory_052018.csv', index=False)

In [ ]:
customers_red_mem.to_csv('Udacity_CUSTOMERS_reduced_memory_052018.csv', index=False)

### Start

In [3]:
%%time
# Read files with reduced memory
azdias = pd.read_csv('Udacity_AZDIAS_reduced_memory_052018.csv')

<string>:2: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 22 s, sys: 8.94 s, total: 31 s
Wall time: 32.8 s


In [4]:
%%time
customers = pd.read_csv('Udacity_CUSTOMERS_reduced_memory_052018.csv')

<string>:2: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 4.37 s, sys: 657 ms, total: 5.02 s
Wall time: 5.09 s


In [5]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [6]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [7]:
azdias.shape

(891221, 366)

In [8]:
customers.shape

(191652, 369)

In [9]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


### Fixing the warning messages

In [11]:
# Read information on the attributes 
attributes_vals = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header=1)
attributes_info = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header=1)

In [12]:
attributes_vals.head(10)

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,NaN,0,no classification possible
2,NaN,NaN,NaN,1,passive elderly
3,NaN,NaN,NaN,2,cultural elderly
4,NaN,NaN,NaN,3,experience-driven elderly
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,NaN,1,< 30 years
7,NaN,NaN,NaN,2,30 - 45 years
8,NaN,NaN,NaN,3,46 - 60 years
9,NaN,NaN,NaN,4,> 60 years


In [13]:
# Let us fill in the null values in the attribute column
attributes_vals["Attribute"] = attributes_vals["Attribute"].ffill()
attributes_vals.head(15)

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,AGER_TYP,NaN,0,no classification possible
2,NaN,AGER_TYP,NaN,1,passive elderly
3,NaN,AGER_TYP,NaN,2,cultural elderly
4,NaN,AGER_TYP,NaN,3,experience-driven elderly
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,ALTERSKATEGORIE_GROB,NaN,1,< 30 years
7,NaN,ALTERSKATEGORIE_GROB,NaN,2,30 - 45 years
8,NaN,ALTERSKATEGORIE_GROB,NaN,3,46 - 60 years
9,NaN,ALTERSKATEGORIE_GROB,NaN,4,> 60 years


In [14]:
attributes_info.head(10)

,Unnamed: 0,Information level,Attribute,Description,Additional notes
0,NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,NaN,ANREDE_KZ,gender,NaN
3,NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
5,NaN,NaN,FINANZ_SPARER,financial typology: money saver,NaN
6,NaN,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
7,NaN,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
8,NaN,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
9,NaN,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


In [15]:
attributes_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2258 entries, 0 to 2257
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   0 non-null      float64
 1   Attribute    2258 non-null   object 
 2   Description  351 non-null    object 
 3   Value        2258 non-null   object 
 4   Meaning      2247 non-null   object 
dtypes: float64(1), object(4)
memory usage: 88.3+ KB


In [16]:
attributes_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         0 non-null      float64
 1   Information level  10 non-null     object 
 2   Attribute          313 non-null    object 
 3   Description        313 non-null    object 
 4   Additional notes   20 non-null     object 
dtypes: float64(1), object(4)
memory usage: 12.4+ KB


### 0.1 Data Cleaning

#### Verify Integrity

In [ ]:
# Print the shapes for the dataframes
print("Azdias shape:", azdias.shape)
print("Customers shape:", customers.shape)

Let's check the number of null values

In [ ]:
azdias.isnull().sum().sum()

In [ ]:
customers.isnull().sum().sum()

In [ ]:
# Print the percentage of missing values for each dataset
print("Percentage of missing values for Azdias:", 
      azdias.isnull().sum().sum()*100/(azdias.shape[0]*azdias.shape[1]))
print("Percentage of missing values for Customers:",
      customers.isnull().sum().sum()*100/(customers.shape[0]*customers.shape[1]))

Let's check the number of duplicated values

In [ ]:
azdias.duplicated().sum().sum()

In [ ]:
customers.duplicated().sum().sum()

There are no duplicated rows in two datasets.

It seems that a dataframe with customers'data has 3 extra columns.

In [17]:
# Check the extra columns
set_diff = set(list(customers.columns)) - set(list(azdias.columns))
extra_cols = list(set_diff)
print("Extra columns in Customers dataset:", extra_cols)

Extra columns in Customers dataset: ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']


In [18]:
customers[extra_cols].head()

,CUSTOMER_GROUP,ONLINE_PURCHASE,PRODUCT_GROUP
0,MULTI_BUYER,0,COSMETIC_AND_FOOD
1,SINGLE_BUYER,0,FOOD
2,MULTI_BUYER,0,COSMETIC_AND_FOOD
3,MULTI_BUYER,0,COSMETIC
4,MULTI_BUYER,0,FOOD


#### Fixing warning messages

In [19]:
# Let's have a look at the columns with warnings
azdias.columns[18:20]

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')

Let us check those values in a dataframe with attribute values

In [20]:
attributes_vals[attributes_vals["Attribute"] == "CAMEO_DEUG_2015"]

,Unnamed: 0,Attribute,Description,Value,Meaning
51,NaN,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
52,NaN,CAMEO_DEUG_2015,NaN,1,upper class
53,NaN,CAMEO_DEUG_2015,NaN,2,upper middleclass
54,NaN,CAMEO_DEUG_2015,NaN,3,established middleclasse
55,NaN,CAMEO_DEUG_2015,NaN,4,consumption-oriented middleclass
56,NaN,CAMEO_DEUG_2015,NaN,5,active middleclass
57,NaN,CAMEO_DEUG_2015,NaN,6,low-consumption middleclass
58,NaN,CAMEO_DEUG_2015,NaN,7,lower middleclass
59,NaN,CAMEO_DEUG_2015,NaN,8,working class
60,NaN,CAMEO_DEUG_2015,NaN,9,urban working class


In [21]:
attributes_vals[attributes_vals["Attribute"] == "CAMEO_DEUINTL_2015"]

,Unnamed: 0,Attribute,Description,Value,Meaning
105,NaN,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
106,NaN,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
107,NaN,CAMEO_DEUINTL_2015,NaN,12,Wealthy Households-Young Couples With Children
108,NaN,CAMEO_DEUINTL_2015,NaN,13,Wealthy Households-Families With School Age Ch...
109,NaN,CAMEO_DEUINTL_2015,NaN,14,Wealthy Households-Older Families & Mature Co...
110,NaN,CAMEO_DEUINTL_2015,NaN,15,Wealthy Households-Elders In Retirement
111,NaN,CAMEO_DEUINTL_2015,NaN,21,Prosperous Households-Pre-Family Couples & Sin...
112,NaN,CAMEO_DEUINTL_2015,NaN,22,Prosperous Households-Young Couples With Children
113,NaN,CAMEO_DEUINTL_2015,NaN,23,Prosperous Households-Families With School Age...
114,NaN,CAMEO_DEUINTL_2015,NaN,24,Prosperous Households-Older Families & Mature ...


Based on the attribute values dataframe, it's clear that the values should be integer.

In [22]:
warning_columns = list(azdias.columns[18:20])

In [23]:
azdias[warning_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   CAMEO_DEUG_2015  792242 non-null  object
 1   CAMEO_INTL_2015  792242 non-null  object
dtypes: object(2)
memory usage: 13.6+ MB


The warning columns have object type instead of integer.

In [ ]:
# Let's have a closer look at the values in the warning columns
azdias[warning_columns[0]].unique()

In [ ]:
azdias[warning_columns[1]].unique()

In [ ]:
customers[warning_columns[0]].unique()

In [ ]:
customers[warning_columns[1]].unique()

The solumns contain nan, 'XX' and 'X' values, which are not in the attribute list of values. Therefore, let's replace thoose values by -1, which means unknown information.

In [24]:
def fix_warnings(df):
    df.replace({warning_columns[0]: {'X': -1, 'XX': -1}}, inplace=True)
    df.replace({warning_columns[1]: {'X': -1, 'XX': -1}}, inplace=True)
    df[warning_columns[0]].fillna(-1, inplace=True)
    df[warning_columns[1]].fillna(-1, inplace=True)
    df[warning_columns] = df[warning_columns].astype(float)
    return df

In [25]:
azdias = fix_warnings(azdias)
customers = fix_warnings(customers)

In [26]:
# Let's have a closer look at the values in the warning columns
azdias[warning_columns[0]].value_counts()

 8.0    134441
 9.0    108177
 6.0    105874
 4.0    103912
-1.0     99352
 3.0     86779
 2.0     83231
 7.0     77933
 5.0     55310
 1.0     36212
Name: CAMEO_DEUG_2015, dtype: int64

In [27]:
# Let's have a closer look at the values in the warning columns
azdias[warning_columns[1]].value_counts()

 51.0    133694
-1.0      99352
 41.0     92336
 24.0     91158
 14.0     62884
 43.0     56672
 54.0     45391
 25.0     39628
 22.0     33155
 23.0     26750
 13.0     26336
 45.0     26132
 55.0     23955
 52.0     20542
 31.0     19024
 34.0     18524
 15.0     16974
 44.0     14820
 12.0     13249
 35.0     10356
 32.0     10354
 33.0      9935
Name: CAMEO_INTL_2015, dtype: int64

In [28]:
customers[warning_columns[0]].value_counts()

-1.0    50554
 2.0    23484
 4.0    22064
 6.0    18717
 3.0    18390
 1.0    16778
 8.0    13049
 5.0    11666
 7.0    10558
 9.0     6392
Name: CAMEO_DEUG_2015, dtype: int64

In [29]:
customers[warning_columns[1]].value_counts()

-1.0     50554
 14.0    19647
 24.0    17805
 41.0    11320
 43.0     9634
 25.0     9372
 15.0     9217
 51.0     8113
 13.0     7683
 22.0     7507
 23.0     5770
 34.0     5368
 45.0     5288
 54.0     4795
 12.0     3715
 55.0     3714
 44.0     2832
 31.0     2731
 35.0     2294
 32.0     1696
 33.0     1574
 52.0     1023
Name: CAMEO_INTL_2015, dtype: int64

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
%%time
profile_azdias = ProfileReport(azdias, title='EDA for Azdias dataset', explorative = True)
profile_azdias

In [ ]:
%%time
profile_customers = ProfileReport(customers, title='EDA for Customers dataset', explorative = True)
profile_customers

In [ ]:
%%time
# Make output files
profile_azdias.to_file(output_file='output_azdias.html')

In [ ]:
%%time
profile_customers.to_file(output_file='output_customers.html')

### Compare Azdias and Customers dataframes with attribute values [failed to do]

In [ ]:
attributes_vals

In [ ]:
# Create a dictionary with attributes and its values
attributes_dict_min = attributes_vals.groupby('Attribute')['Value'].unique().to_dict()

In [ ]:
#attributes_df

In [ ]:
azdias_cols = azdias.columns
customers_cols = customers.columns

In [ ]:
# Create dictionaries without values
azdias_dict = {key: None for key in azdias_cols} 
customers_dict = {key: None for key in customers_cols}

In [ ]:
azdias_dict = dict(zip([i for i in azdias.columns] , [azdias[i].unique() for i in azdias.columns]))
customers_dict = dict(zip([i for i in customers.columns] , [customers[i].unique() for i in customers.columns]))

In [ ]:
azdias_diff = { k : azdias_dict[k] for k in set(azdias_dict) - set(attributes_dict) }

In [ ]:
azdias.head()

In [ ]:
customers.head()

In [ ]:
diffazdias = DeepDiff(azdias_dict, attributes_dict, ignore_order=True)

In [ ]:
diff_azdias

### Unpivot dataframes

In [30]:
%%time
azdias_unpivoted = azdias.melt(id_vars=['LNR'], var_name='Attribute', value_name='Value')

CPU times: user 37.1 s, sys: 1min 44s, total: 2min 21s
Wall time: 3min 38s


In [31]:
azdias_unpivoted.head()

,LNR,Attribute,Value
0,910215,AGER_TYP,-1
1,910220,AGER_TYP,-1
2,910225,AGER_TYP,-1
3,910226,AGER_TYP,2
4,910241,AGER_TYP,-1


In [36]:
azdias_unpivoted.shape

(325295665, 3)

In [37]:
azdias_unpivoted.drop(columns=['LNR'], inplace=True)

In [39]:
azdias_attr = azdias_unpivoted.drop_duplicates(subset=['Attribute', 'Value'], keep='last')

In [52]:
azdias_attr.head(15)

,Attribute,Value
891157,AGER_TYP,3
891178,AGER_TYP,0
891195,AGER_TYP,2
891207,AGER_TYP,1
891220,AGER_TYP,-1
1782376,AKT_DAT_KL,4
1782377,AKT_DAT_KL,3
1782389,AKT_DAT_KL,7
1782394,AKT_DAT_KL,6
1782402,AKT_DAT_KL,8


In [41]:
azdias_attr.shape

(9957, 2)

In [42]:
# Save results
azdias_attr.to_csv('azdias_attributes_before_cleaning.csv', index=False)

In [43]:
del azdias_unpivoted

In [44]:
%%time
# unpivot customers dataframe
customers_unpivoted = customers.melt(id_vars=['LNR'], var_name='Attribute', value_name='Value')

CPU times: user 6.22 s, sys: 6.82 s, total: 13 s
Wall time: 15 s


In [45]:
customers_unpivoted.head()

,LNR,Attribute,Value
0,9626,AGER_TYP,2
1,9628,AGER_TYP,-1
2,143872,AGER_TYP,-1
3,143873,AGER_TYP,1
4,143874,AGER_TYP,-1


In [46]:
customers_unpivoted.shape

(70527936, 3)

In [47]:
customers_unpivoted.drop(columns=['LNR'], inplace=True)

In [48]:
customers_attr = customers_unpivoted.drop_duplicates(subset=['Attribute', 'Value'], keep='last')

In [54]:
customers_attr.head(15)

,Attribute,Value
191632,AGER_TYP,0
191647,AGER_TYP,1
191648,AGER_TYP,-1
191649,AGER_TYP,2
191651,AGER_TYP,3
382958,AKT_DAT_KL,6
383133,AKT_DAT_KL,NaN
383162,AKT_DAT_KL,8
383231,AKT_DAT_KL,5
383249,AKT_DAT_KL,4


In [50]:
customers_attr.shape

(7652, 2)

In [51]:
# Save customer attributes
customers_attr.to_csv('customers_attributes_before_cleaning.csv', index=False)

In [53]:
del customers_unpivoted

### Compare with attributes values

In [55]:
attributes_vals.head()

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,AGER_TYP,NaN,0,no classification possible
2,NaN,AGER_TYP,NaN,1,passive elderly
3,NaN,AGER_TYP,NaN,2,cultural elderly
4,NaN,AGER_TYP,NaN,3,experience-driven elderly


In [85]:
attributes = attributes_vals.drop(columns=['Unnamed: 0', 'Description', 'Meaning'])
attributes.head()

,Attribute,Value
0,AGER_TYP,-1
1,AGER_TYP,0
2,AGER_TYP,1
3,AGER_TYP,2
4,AGER_TYP,3


### Inspect obtained dataframes with attributes using pandas profiling

In [74]:
from pandas_profiling import ProfileReport

In [75]:
%%time
profile_azdias_attr = ProfileReport(azdias_attr, title='EDA for Azdias attributes', explorative = True)
profile_azdias_attr

CPU times: user 34.8 ms, sys: 59.1 ms, total: 93.9 ms
Wall time: 191 ms


In [76]:
%%time
# Make output files
profile_azdias_attr.to_file(output_file='azdias_attributes.html')


CPU times: user 2.38 s, sys: 338 ms, total: 2.71 s
Wall time: 5.79 s


In [77]:
%%time
profile_customers_attr = ProfileReport(customers_attr, title='EDA for Customers attributes', explorative = True)
profile_customers_attr

CPU times: user 34.5 ms, sys: 9.2 ms, total: 43.7 ms
Wall time: 47.6 ms


In [78]:
%%time
# Make output files
profile_customers_attr.to_file(output_file='customers_attributes.html')


CPU times: user 1.83 s, sys: 100 ms, total: 1.93 s
Wall time: 1.97 s


In [83]:
%%time
profile_attr_vals = ProfileReport(attributes_vals, title='EDA for Attribute values', explorative = True)
profile_attr_vals

CPU times: user 37.2 ms, sys: 1.7 ms, total: 38.9 ms
Wall time: 37.6 ms


In [84]:
%%time
# Make output files
profile_attr_vals.to_file(output_file='attributes_values.html')


CPU times: user 2.3 s, sys: 107 ms, total: 2.41 s
Wall time: 2.5 s


### Check data types

In [93]:
attributes.dtypes

Attribute            object
Value                object
Defined_attribute     int64
dtype: object

In [94]:
azdias_attr.dtypes

Attribute    object
Value        object
dtype: object

In [96]:
customers_attr.dtypes

Attribute    object
Value        object
dtype: object

It seems 'Attribut'' and 'Value'columns have the same data type

In [97]:
attributes['Defined_attribute'] = 1

In [98]:
azdias_attributes = pd.merge(azdias_attr, attributes, how='left', on=['Attribute', 'Value'])

In [99]:
customers_attributes = pd.merge(customers_attr, attributes, how='left', on=['Attribute', 'Value'])

In [100]:
# Fill in NULL values
azdias_attributes['Defined_attribute'].fillna(0, inplace=True)
customers_attributes['Defined_attribute'].fillna(0, inplace=True)

In [101]:
azdias_attributes['Defined_attribute'].value_counts()

0.0    8359
1.0    1598
Name: Defined_attribute, dtype: int64

In [102]:
customers_attributes['Defined_attribute'].value_counts()

0.0    6064
1.0    1588
Name: Defined_attribute, dtype: int64

In [107]:
azdias_check = azdias_attributes[azdias_attributes['Defined_attribute']==0]
azdias_check

,Attribute,Value,Defined_attribute
5,AKT_DAT_KL,4,0.0
6,AKT_DAT_KL,3,0.0
7,AKT_DAT_KL,7,0.0
8,AKT_DAT_KL,6,0.0
9,AKT_DAT_KL,8,0.0
...,...,...,...
9916,VK_ZG11,7,0.0
9918,W_KEIT_KIND_HH,0,0.0
9922,W_KEIT_KIND_HH,NaN,0.0
9927,WOHNDAUER_2008,NaN,0.0


In [109]:
attr_lst = list(attributes['Attribute'].unique())

It seems some attributes are not listed in attribute values xcel file

In [ ]:
%%time
customers_attr = pd.DataFrame(customers_unpivoted.groupby('Attribute')['Value'].unique())
customers_attr.reset_index(name='Value')

In [ ]:
customers_attr.head()

In [ ]:
customers_attr.to_csv('customers_attributes_before_cleaning.csv', index=False)

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')